[View in Colaboratory](https://colab.research.google.com/github/upadhyayprakash/EIP/blob/master/4B_5_Try_5_Epochs.ipynb)

In [0]:
# https://keras.io/
!pip install -q keras
import keras

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 5
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# Added by Prakash
#x_train = x_train.astype('float32')
#x_test = x_test.astype('float32')
#x_train /= 255
#x_test /= 255

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    #AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    #flat = Flatten()(AvgPooling)
    flat = Flatten()(relu)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 12
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

# START: Added Pooling & Dropout by Prakash
#BatchNorm = BatchNormalization()(input)
#relu = Activation('relu')(BatchNorm)
    
#Second_Conv2D = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)

#DropOut_Conv2D = Dropout(0.2)(Second_Conv2D)
#AvgPooling2D_Conv2D = AveragePooling2D(pool_size=(2,2))(DropOut_Conv2D)

# END: Added Pooling & Dropout by Prakash

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
#Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

#Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
#Last_Block = add_denseblock(Third_Block,  num_filter, dropout_rate)

output = output_layer(Third_Block)

In [159]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_521 (Conv2D)             (None, 32, 32, 12)   324         input_12[0][0]                   
__________________________________________________________________________________________________
batch_normalization_521 (BatchN (None, 32, 32, 12)   48          conv2d_521[0][0]                 
__________________________________________________________________________________________________
activation_521 (Activation)     (None, 32, 32, 12)   0           batch_normalization_521[0][0]    
__________________________________________________________________________________________________
conv2d_522

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [161]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 179s 4ms/step - loss: 1.6711 - acc: 0.3977 - val_loss: 1.5799 - val_acc: 0.4622
Epoch 2/5
19456/50000 [==========>...................] - ETA: 1:24 - loss: 1.3619 - acc: 0.5076

50000/50000 [==============================] - 147s 3ms/step - loss: 1.3009 - acc: 0.5310 - val_loss: 1.6273 - val_acc: 0.4458
Epoch 3/5
45952/50000 [==========================>...] - ETA: 11s - loss: 1.1489 - acc: 0.5895

50000/50000 [==============================] - 147s 3ms/step - loss: 1.1444 - acc: 0.5913 - val_loss: 1.1478 - val_acc: 0.6018
Epoch 4/5
50000/50000 [==============================] - 147s 3ms/step - loss: 1.0371 - acc: 0.6306 - val_loss: 1.0680 - val_acc: 0.6320
Epoch 5/5
 3584/50000 [=>............................] - ETA: 2:07 - loss: 0.9519 - acc: 0.6669

50000/50000 [==============================] - 147s 3ms/step - loss: 0.9477 - acc: 0.6642 - val_loss: 1.0452 - val_acc: 0.6468


In [162]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 14s 1ms/step
Test loss: 1.0451789587020874
Test accuracy: 0.6468
